In [1]:
%run C:/__PYP/__COMMON/ugenejupyterlib/firstcell.ipynb
%run C:/__PYP/__COMMON/ugenejupyterlib/datacell.ipynb

@
        # platform.node()='MH507192' 
        # datetime.now()=datetime.datetime(2020, 11, 13, 8, 30, 49, 180036) 
        # sys.version='3.8.3 (default, Jul  2 2020, 17:30:36) [MSC v.1916 64 bit (AMD64)]' 
        


### snipets to test your knowledge in pd's :
### assign, groupby

In [2]:
# the problem to be solved:
ref = pd.DataFrame({'l_idx': [[0, 2, 3], [3, 4, 6], [1,2,3,6]]}, index='g0 g1 g2'.split())
df = pd.DataFrame({ 'o1': ['A1', 'B1', 'A1', 'C1', 'D1', 'E1', 'F1'], 'o2': list('abcdefg')})
# df.rename(index={None:'idx'}); df.rename(index={'index':'idx'}) # won't work as index doesn't have a name by default, can't design
df.index.name = 'idx'

s = ref['l_idx'].explode()
desired_output= ref.join(df.loc[s].groupby(s.index).agg(lambda x: list(x)))

printdf('ref')
printdf('df')
"to be combined into:"
desired_output

[df] ref:
+----+--------------+
|    | l_idx        |
|----+--------------|
| g0 | [0, 2, 3]    |
| g1 | [3, 4, 6]    |
| g2 | [1, 2, 3, 6] |
+----+--------------+

[df] df:
+-------+------+------+
|   idx | o1   | o2   |
|-------+------+------|
|     0 | A1   | a    |
|     1 | B1   | b    |
|     2 | A1   | c    |
|     3 | C1   | d    |
|     4 | D1   | e    |
|     5 | E1   | f    |
|     6 | F1   | g    |
+-------+------+------+



'to be combined into:'

,l_idx,o1,o2
g0,"[0, 2, 3]","[A1, A1, C1]","[a, c, d]"
g1,"[3, 4, 6]","[C1, D1, F1]","[d, e, g]"
g2,"[1, 2, 3, 6]","[B1, A1, C1, F1]","[b, c, d, g]"


In [3]:
# [+]  df explode, for cells whoes values are not single value, but a list

m = ref['l_idx'].explode() # m as in "mapping"
help(printdf)
printdf('m')

Help on function printdf in module __main__:

printdf(code, dfname: str = None)
    example: printdf(mydf, 'mydf'); printdf(mydf); printdf('mydf')
    @para code: could be df, sr, or string

[sr] m:
+----+---------+
|    |   l_idx |
|----+---------|
| g0 |       0 |
| g0 |       2 |
| g0 |       3 |
| g1 |       3 |
| g1 |       4 |
| g1 |       6 |
| g2 |       1 |
| g2 |       2 |
| g2 |       3 |
| g2 |       6 |
+----+---------+



In [4]:
# [+] reverse sr m
rm = m.to_frame().reset_index()
vals = rm['l_idx'].values
rm['l_idx'] = vals[::-1]
rm = rm.set_index('l_idx')
rm.index.name=None

printdf(rm, 'reversed m, rm')

[df] reversed m, rm:
+----+---------+
|    | index   |
|----+---------|
|  6 | g0      |
|  3 | g0      |
|  2 | g0      |
|  1 | g1      |
|  6 | g1      |
|  4 | g1      |
|  3 | g2      |
|  3 | g2      |
|  2 | g2      |
|  0 | g2      |
+----+---------+



In [5]:
# groupby review
simpleMapping = {0:'g0', 1:'g1', 2:'g1', 3:'g1'}

printgp(df.groupby(simpleMapping))

# we know that groupby's mapping is to map index into groups
# a thing we didn't emphacise before:
# each index can only be mapped to ONLY ONE group
# thinking about the dictionary or Sr or df.col, they are all dictionary based, so the mapping is USUALLY** unique
# however sr of df.col CAN support NON UNIQUE index. then you will have errors, as shown in the next cell

[df] [group] g0:
+-------+------+------+
|   idx | o1   | o2   |
|-------+------+------|
|     0 | A1   | a    |
+-------+------+------+

[df] [group] g1:
+-------+------+------+
|   idx | o1   | o2   |
|-------+------+------|
|     1 | B1   | b    |
|     2 | A1   | c    |
|     3 | C1   | d    |
+-------+------+------+



In [6]:
# WRONG SOLUTIONS
# !!! very important knowledge !!!

try:
    gp = df.o1[m.values].groupby(rm)
except Exception as e:
    print("df.o1[m.values].groupby(rm) ==>\n", "!!EXCEPTION!!", e)
ccc=\
"""
Grouper for '<class 'pandas.core.frame.DataFrame'>' not 1-dimensional
the error message is obtuse, 
usually it happens when you df.groupby(anotherdf), but anotherdf has more than 1 column
what's happening is, in anotherdf, an index is mapped to more than 1 labels,
ie, index 0 is mapped to column "fruit", cell value 'apple'; index 0 is also mapped to 2nd column "organic", cell value True

in this case, we don't have a multi dimensional column, but we have a lookup mapping whoes index is not unique
"""

try:
    df.o1[m.values].groupby(m)
except Exception as e:
    print('df.o1[m.values].groupby(m) ==>\n', "!!EXCEPTION!!", e)
    # same ballpark explanation as above
    
try:
    gp = df.o1[m.values].groupby(rm['index'])
except Exception as e:
    print("df.o1[m.values].groupby(rm['index']) ==>\n", "!!EXCEPTION!!", e)
    # same ballpark explanation as above    


df.o1[m.values].groupby(rm) ==>
 !!EXCEPTION!! Grouper for '<class 'pandas.core.frame.DataFrame'>' not 1-dimensional
df.o1[m.values].groupby(m) ==>
 !!EXCEPTION!! cannot reindex from a duplicate axis
df.o1[m.values].groupby(rm['index']) ==>
 !!EXCEPTION!! cannot reindex from a duplicate axis


In [7]:
# GROUPBY() using 1D List, ie, no lookup
# this problem feels like groupby(), the ref's cell contains "grouping" or list of indexes
# but we can't really groupby, as the index is not unique after explode()

ccc=\
"""
THIS COMES DOWN TO ANOTHER WAY OF GROUPBY(). GROUPBY() CAN TAKE A LIST AS A MAPPING
IN THAT CASE, 
    LIST IS CONSIDERED THE LABEL, 
    "index" is implied to be list's element's sequence. "index" will the thing we will be grouping
    "index" is not actual df.groupby()'s index, is more like iat(), absolute position
"""
temp = pd.DataFrame(data='notused', index=list(range(11, 39, 3)), columns=['notused'])
printdf('temp')
printdf('m')

# m.index is a list, used as lookup map to groupby temp, which has none natural squence index.
printgp(    temp.groupby(m.index)  )

[df] temp:
+----+-----------+
|    | notused   |
|----+-----------|
| 11 | notused   |
| 14 | notused   |
| 17 | notused   |
| 20 | notused   |
| 23 | notused   |
| 26 | notused   |
| 29 | notused   |
| 32 | notused   |
| 35 | notused   |
| 38 | notused   |
+----+-----------+

[sr] m:
+----+---------+
|    |   l_idx |
|----+---------|
| g0 |       0 |
| g0 |       2 |
| g0 |       3 |
| g1 |       3 |
| g1 |       4 |
| g1 |       6 |
| g2 |       1 |
| g2 |       2 |
| g2 |       3 |
| g2 |       6 |
+----+---------+

[df] [group] g0:
+----+-----------+
|    | notused   |
|----+-----------|
| 11 | notused   |
| 14 | notused   |
| 17 | notused   |
+----+-----------+

[df] [group] g1:
+----+-----------+
|    | notused   |
|----+-----------|
| 20 | notused   |
| 23 | notused   |
| 26 | notused   |
+----+-----------+

[df] [group] g2:
+----+-----------+
|    | notused   |
|----+-----------|
| 29 | notused   |
| 32 | notused   |
| 35 | notused   |
| 38 | notused   |
+----+-----------+



In [20]:
type(
    df.loc[:, 'o1']
)
df.index==2
df
df.o2=='a'

if df:
    pass

pandas.core.series.Series

array([False, False,  True, False, False, False, False])

,o1,o2
0,A1,a
1,B1,b
2,A1,c
3,C1,d
4,D1,e
5,E1,f
6,F1,g


0     True
1    False
2    False
3    False
4    False
5    False
6    False
Name: o2, dtype: bool

ValueError: The truth value of a DataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [10]:
# final solution:

ref = pd.DataFrame({'l_idx': [[0, 2, 3], [3, 4, 6], [1,2,3,6]]}, index='g0 g1 g2'.split())
df = pd.DataFrame({ 'o1': ['A1', 'B1', 'A1', 'C1', 'D1', 'E1', 'F1'], 'o2': list('abcdefg')})
printdf('ref')
printdf('df')

m = ref['l_idx'].explode()
m 

[df] ref:
+----+--------------+
|    | l_idx        |
|----+--------------|
| g0 | [0, 2, 3]    |
| g1 | [3, 4, 6]    |
| g2 | [1, 2, 3, 6] |
+----+--------------+

[df] df:
+----+------+------+
|    | o1   | o2   |
|----+------+------|
|  0 | A1   | a    |
|  1 | B1   | b    |
|  2 | A1   | c    |
|  3 | C1   | d    |
|  4 | D1   | e    |
|  5 | E1   | f    |
|  6 | F1   | g    |
+----+------+------+



g0    0
g0    2
g0    3
g1    3
g1    4
g1    6
g2    1
g2    2
g2    3
g2    6
Name: l_idx, dtype: object